### Merging multiple model outputs into one ensemble dataset

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import xarray as xr
import nc_time_axis
import pandas as pd
import re
import cftime
from tqdm import tqdm
import os
import warnings

In [116]:
MODEL_DIR = "../data/models"
ds_paths = []
for model in os.listdir(MODEL_DIR):
    for member in os.listdir(os.path.join(MODEL_DIR, model, "psl")):
        ds_paths.append(os.path.join(MODEL_DIR, model, "psl", member))

# Add all the datasets into a list for concatenation
ds_list = []
interp_ds_list = []

# Suppress the runtime warnings against calendar conversions
with warnings.catch_warnings(action="ignore"):
    # Use the example dataset to interpolate every other
    example_ds = xr.open_dataset(ds_paths[0])
    example_ds['time'] = example_ds.indexes['time'].to_datetimeindex()
    
    for i in tqdm(range(len(ds_paths)), desc=""):
        ds = xr.open_dataset(ds_paths[i])
        ds_list.append(ds)
        # For the GFDL-CM2.1 model, drop the variables that we don't need
        ds = ds.drop_vars(["lon_bnds", "lat_bnds", "time_bnds"])
        
        if "average_DT" in ds.variables:
            ds = ds.drop_vars(["average_DT", "average_T1", "average_T2"])
        
        if type(ds.indexes['time'])!= pd.core.indexes.datetimes.DatetimeIndex:
            ds['time'] = ds.indexes['time'].to_datetimeindex()
        else:
            ds['time'] = ds.indexes['time']
        ds_new = ds.interp_like(example_ds)
        interp_ds_list.append(ds_new)
        
ensemble = xr.concat(interp_ds_list, dim="n")

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:06<00:00,  7.73it/s]


In [118]:
ensemble.to_netcdf("../data/ensembles/large_psl_decadal_ensemble_no_extrapolate.nc")